In [1]:
import sys
import os
import numpy as np
import pandas as pd
import cv2
#import tensorflow as tf
import glob
import pcl
import math
import pickle
import six
import parse_xml as pt


In [2]:
import traceback
def pv(var):
    (filename,line_number,function_name,text)=traceback.extract_stack()[-2]
    print('%s: %s'%(text[text.find('(')+1:-1],var))


In [73]:
#os.environ["CUDA_VISIBLE_DEVICES"] = ""
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import model_01_conv
import layers
import input_helpers


In [74]:
# reload everything

import imp 

imp.reload(input_helpers)
imp.reload(layers)
imp.reload(model_01_conv)
layers.cnn_model = None
None

In [75]:
# Clear graph
try:
    with model_01_conv.tf.Session() as sess:
        #with model_01_conv.tf.device("/cpu:0"):
        model_01_conv.tf.reset_default_graph()
    None
except:
    pass

In [76]:
resolution=0.1,
scale=8,
voxel_shape=(800, 800, 80),
x=(-40, 40),
y=(-40, 40),
z=(-4, 4),

In [77]:
model_name = "3dcnn_800x800x80_res0.1_sc86.ckpt"
model_path = "/home/paperspace/Desktop/didi_solution/cnn_3d/"+model_name

model = model_01_conv.CNNModel(resolution=0.1,
                                scale=8,
                                voxel_shape=(800, 800, 80),
                                x=(-40, 40),
                                y=(-40, 40),
                                z=(-4, 4),
                                is_training=None,
                                model_path=model_path
                               )

model.build_graph()


INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt


In [103]:
import conversion_helpers
from collections import defaultdict

def make_frame_result_dict_shell():
    return {'pred': {}, 'actual': {}}

test_folder_glob = "/home/paperspace/Desktop/converted/car/testing/*"


for test_folder in glob.glob(test_folder_glob):
    bag_name = test_folder.split('/')[-1]
    if bag_name != "ford03":
        continue
    output_dir = os.path.join(os.getcwd(), 'test_output', bag_name)
    conversion_helpers.mkdir_p(output_dir)
    pcd_glob = os.path.join(test_folder, 'points', '*')
    
    print("processing: '%s'"%pcd_glob)
    results = defaultdict(make_frame_result_dict_shell)
    
    for point_path in glob.glob(pcd_glob):
        frame_id = point_path.split('/')[-1].split('.')[0]
        
        pc = input_helpers.load_pc_from_pcd(point_path)
        
        corners, centers, rotations, confidence = model.predict(pc)
        results[frame_id]['pred'] = {
            'corners': corners,
            'centers': centers,
            'confidence': confidence,
            'rot': rotations
        }

    pickle_path = os.path.join(output_dir, "results.pickle")
    print("saving to: '%s'"%pickle_path) 
    pickle.dump(dict(results), open(pickle_path, "wb") )
    
    print(" ")

    
    

processing: '/home/paperspace/Desktop/converted/car/testing/ford03/points/*'
saving to: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford03/results.pickle'
 


In [109]:
from tracklets.python.generate_tracklet import Tracklet, TrackletCollection


In [141]:
def get_orientation(corners_initial):
    corners_initial = np.reshape(corners_initial, (-1, 3))
    average = np.sum(corners_initial, axis=0) / corners_initial.shape[0]
    corners = corners_initial - average
    convx_hull = np.reshape(cv2.convexHull(np.array(corners[:, :2], dtype=np.float32)), (-1, 2))
    rotated_convx_hull = np.vstack((convx_hull[1:], convx_hull[:1]))
    distance_delta = convx_hull - rotated_convx_hull
    distance_delta = np.linalg.norm(convx_hull - rotated_convx_hull, axis=1)
    length_of_car_from_convex_hull = np.max(distance_delta)
    width_of_car_from_convex_hull = np.min(distance_delta)
    l, w = length_of_car_from_convex_hull, width_of_car_from_convex_hull
    a_mat = [[-l / 2., -w / 2.],
            [l / 2., -w / 2.],
            [-l / 2., w / 2.],
            [-l / 2., -w / 2.],
            [-l / 2., w / 2.],
            [l / 2., w / 2.],
            [l / 2., -w / 2.],
            [l / 2., w / 2.]]
    b_mat = corners[:, :2]
    rot_mat = np.linalg.lstsq(a_mat, b_mat)[0]
    angle = math.acos(rot_mat[0][0])
    h = np.max(corners_initial[:, 2]) - np.min(corners_initial[:, 2])
    return angle, l, w, h

In [163]:
results_folder_glob = "/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/*"
xml_output_path = "/home/paperspace/Desktop/didi_solution/cnn_3d/tracklet_output/"

for results_folder in glob.glob(results_folder_glob):
    bag_name = results_folder.split('/')[-1]
    results_pickle_path = os.path.join(results_folder, "results.pickle")
    print("processing: '%s'"%results_pickle_path)
    
    tc = TrackletCollection()
    results = pickle.load(open(results_pickle_path, "rb") )
    frame_ids = sorted(list(map(int, results.keys())))
    
    #if "mustang" in bag_name:
    h = 1.481
    w = 1.916
    l = 4.785
        
    #else:
    #    h = 1.6510
    #    w = 1.7780
    #    l = 4.5212

    poses = []
    last_pose = None
    all_corners = []
    starting_frame = 0
    for frame_id in range(max(frame_ids)):
        frame_results = results.get(str(frame_id), None)

        frame_valid = False
        
        if frame_results:
            pred = frame_results['pred']
            
            if pred['confidence'] is not None and pred['confidence'].shape[0] > 0:
                frame_valid = True
                res_i = np.argmax(pred['confidence'])

                center = pred['centers'][res_i]
                rot = pred['rot'][res_i]
                corners = pred['corners'][res_i]
                    
                last_pose = {
                    "tx": center[0],
                    "ty": center[1],
                    "tz": center[2],
                    "rx": 0.0,
                    "ry": 0.0,
                    "rz": rot
                }

                poses.append(last_pose)

            else:
                frame_valid = False
            
        if not frame_valid:
            if last_pose is None:
                starting_frame += 1
            else:
                if starting_frame > 0:
                    for i in range(starting_frame):
                        poses.append(last_pose)
                        starting_frame = 0
                else:
                    poses.append(last_pose)

    tracklet = Tracklet(object_type="Car", l=l, w=w, h=h)
    for pose in poses:
        tracklet.poses.append(pose)
    tracklet.first_frame = 0
    
    tc.tracklets.append(tracklet)
    tc.write_xml(os.path.join(xml_output_path, bag_name+".xml"))
    
    
    

processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford03/results.pickle'
processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/mustang01/results.pickle'
processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford07/results.pickle'
processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford05/results.pickle'
processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford06/results.pickle'
processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford04/results.pickle'
processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford02/results.pickle'
processing: '/home/paperspace/Desktop/didi_solution/cnn_3d/test_output/ford01/results.pickle'


In [78]:

#path_prefix = "/home/paperspace/Desktop/converted/car/training/suburu_leading_front_left/*/"
#path_prefix = "/home/paperspace/Desktop/converted/pedestrian/5/"
path_prefix = "/home/paperspace/Desktop/converted/car/testing/ford04/"
points_glob = path_prefix+"points/*.pcd"

points_paths = glob.glob(points_glob)
points_path = points_paths[30]
model.build_graph()

folder_path = "/home/paperspace/Desktop/converted/car/testing/*"
pc = input_helpers.load_pc_from_pcd(points_path)


In [79]:
import time

for i in range(5):
    t_start = time.time()
    corners, centers, rotations, confidence = model.predict(pc)
    print("took: %sm"%str((time.time() - t_start)/60.0))    

took: 0.02688827117284139m
took: 0.02916560967763265m
took: 0.029087960720062256m
took: 0.02827640771865845m
took: 0.029466478029886882m


In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import cv2
#import tensorflow as tf
import glob
import pcl
import math
import pickle
import six
import parse_xml as pt
import model_01_conv
import layers
import input_helpers
import traceback
def pv(var):
    (filename,line_number,function_name,text)=traceback.extract_stack()[-2]
    print('%s: %s'%(text[text.find('(')+1:-1],var))


In [5]:

def bring_bbox_to_ground(corners):
    grounded_corners = []
    for corner in corners.reshape(-1, 8, 3):
        corner[:, 2] -= np.max(corner[:, 2])
        grounded_corners.append(corner)
    return np.array(grounded_corners)

In [ ]:
import sensor_msgs.point_cloud2 as pc2
import rospy
from sensor_msgs.msg import PointCloud2
import std_msgs
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point, TransformStamped
import rospy

node = rospy.init_node("pc2_publisher")
pub = rospy.Publisher("/points_raw", PointCloud2, queue_size=1000000)
pub2 = rospy.Publisher("/points_raw1", PointCloud2, queue_size=1000000)
pub3 = rospy.Publisher("/points_raw2", PointCloud2, queue_size=1000000)

model_name = "3dcnn_800x800x80_res0.1_sc86.ckpt"
model_path = "/home/paperspace/Desktop/didi_solution/cnn_3d/"+model_name
min_certainty = 0.90

resolution=0.1
scale=8
voxel_shape=(800, 800, 80)
x=(-40, 40)
y=(-40, 40)
z=(-4, 4)

model = None
voxel = None
graph = None

import tensorflow as tf
sess = tf.Session()

def init_model():
    global voxel
    global model
    global graph
    if model is not None:
        return model, voxel

    with sess.as_default():
        model, voxel, phase_train = model_01_conv.get_model(sess,
                                                              model_01_conv.CNNModel,
                                                              voxel_shape=voxel_shape,
                                                              activation=tf.nn.relu,
                                                              is_training=None)
        saver = tf.train.Saver()
        saver.restore(sess, model_path)
        graph = tf.get_default_graph()
        
        
    return model, voxel

init_model()


def ros_subscriber():
    #rospy.init_node("lidar2image")
    rospy.Subscriber("/velodyne_points", PointCloud2, callback_got_lidar)
    #pub = rospy.Publisher("/points_raw", PointCloud2, queue_size=10)
    
    
def callback_got_lidar(point_cloud):
    global graph 
    
    model, voxel = init_model()
    pc = np.array([[point[0], point[1], point[2]] for point in pc2.read_points(point_cloud)])
    #pv(pc.shape)
    resolution=0.1
    scale=8
    voxel_shape=(800, 800, 80)
    x=(-40, 40)
    y=(-40, 40)
    z=(-4, 4)

    x = np.array(x)
    y = np.array(y)
    z = np.array(z)

    voxel_pt = input_helpers.pc2voxel(pc,
                             resolution=resolution,
                             x=x,
                             y=y,
                             z=z)

    voxel_x = np.array(voxel_pt).reshape(1, voxel_pt.shape[0], voxel_pt.shape[1], voxel_pt.shape[2], 1)
    #pv(voxel_x.shape)
    with graph.as_default():
        with sess.as_default():
            objectness = sess.run(model.objectness, feed_dict={voxel: voxel_x})[0, :, :, :, 0]
            coordinate = sess.run(model.coordinate, feed_dict={voxel: voxel_x})[0]
            y_pred = sess.run(model.y, feed_dict={voxel: voxel_x})[0, :, :, :, 0]

            index = np.where(y_pred >= np.max(y_pred))# >= min_certainty)
            index = np.array(index)

            centers = np.vstack((index[0], np.vstack((index[1], index[2])))).transpose()
            centers = input_helpers.sphere2center(centers,
                                                resolution=resolution,
                                                scale=scale,
                                                min_value=np.array([x[0], y[0], z[0]]))

            corners = coordinate[index[0], index[1], index[2]].reshape(-1, 8, 3)
            corners = np.array([corners[t,:,:] + centers[t,:] for t in range(corners.shape[0])])
            corners = bring_bbox_to_ground(corners)
            rot = get_orientation(corners)
            pv(rot)

            ts = rospy.Time.now()
            header = std_msgs.msg.Header()
            header.stamp = ts
            header.frame_id = "velodyne"
            points = pc2.create_cloud_xyz32(header, corners.reshape(-1, 3))
            pub2.publish(points)



In [ ]:
points_path = "/home/paperspace/Desktop/converted/car/testing/ford01/points/102.pcd"
pc = input_helpers.load_pc_from_pcd(points_path)
ts = rospy.Time.now()
header = std_msgs.msg.Header()
header.stamp = ts
header.frame_id = "velodyne"
pz = pc2.create_cloud_xyz32(header, pc)
callback_got_lidar(pz)

In [ ]:
ros_subscriber()

In [ ]:
def add_i_to_corners(corners):
    i_corners = []
    i=0

    for corner in corners.reshape(-1, 8, 3):
        new_corner = []
        for c in corner:
            new_corner.append([c[0], c[1], c[2], i])
        i+=1
        
        i_corners.append(new_corner)
    return np.array(i_corners)

def bring_bbox_to_ground(corners):
    grounded_corners = []
    for corner in corners.reshape(-1, 8, 3):
        corner[:, 2] -= np.max(corner[:, 2])
        grounded_corners.append(corner)
    return np.array(grounded_corners)

In [7]:
from collections import defaultdict

def make_frame_result_dict_shell():
    return {'pred': {}, 'actual': {}}
results = defaultdict(make_frame_result_dict_shell)

In [ ]:
results = defaultdict(make_frame_result_dict_shell)

def get_orientation(corners_initial):
    corners_initial = np.reshape(corners_initial, (-1, 3))
    average = np.sum(corners_initial, axis=0) / corners_initial.shape[0]
    corners = corners_initial - average
    #print(corners[:, :2])
    convx_hull = np.reshape(cv2.convexHull(np.array(corners[:, :2], dtype=np.float32)), (-1, 2))
    rotated_convx_hull = np.vstack((convx_hull[1:], convx_hull[:1]))
    distance_delta = convx_hull - rotated_convx_hull
    distance_delta = np.linalg.norm(convx_hull - rotated_convx_hull, axis=1)
    length_of_car_from_convex_hull = np.max(distance_delta)
    width_of_car_from_convex_hull = np.min(distance_delta)
    #print ('length of car:', length_of_car_from_convex_hull, 'width of car:', width_of_car_from_convex_hull)
    l, w = length_of_car_from_convex_hull, width_of_car_from_convex_hull
    a_mat = [[-l / 2., -w / 2.],
            [l / 2., -w / 2.],
            [-l / 2., w / 2.],
            [-l / 2., -w / 2.],
            [-l / 2., w / 2.],
            [l / 2., w / 2.],
            [l / 2., -w / 2.],
            [l / 2., w / 2.]]
    b_mat = corners[:, :2]
    #pv(corners)
    #pv(a_mat)
    #pv(b_mat)
    rot_mat = np.linalg.lstsq(a_mat, b_mat)[0]
    angle = math.acos(rot_mat[0][0])
    h = np.max(corners_initial[:, 2]) - np.min(corners_initial[:, 2])
    return angle, l, w, h



import pickle

model_name = "3dcnn_800x800x80_res0.1_sc86.ckpt"
model_path = "/home/paperspace/Desktop/didi_solution/cnn_3d/"+model_name


path_prefix = "/home/paperspace/Desktop/converted/car/testing/ford01"
#path_prefix = "/home/paperspace/Desktop/converted/car/training/nissan_driving_past_it/*/"
points_glob = path_prefix+"/points/*.pcd"


i = 0
for points_path in glob.glob(points_glob):
    #if i == 10:
    #    break
    #i+=1
        
    frame_id = points_path.split('/')[-1].split('.')[0]
    pv(frame_id)

    coord, objectness, centers, y_pred, confidence = model_01_conv.test(model_path, points_path,
                      voxel_shape=(800, 800, 80),
                      resolution=0.1,
                      scale=8, 
                      x=(-40, 40),
                      y=(-40, 40),
                      z=(-4, 4),
                      min_certainty=0.95
                     )

    index = np.where(y_pred >= np.max(y_pred)) #0.99)#np.max(y_pred))
    index = np.array(index)

    centers = np.vstack((index[0], np.vstack((index[1], index[2])))).transpose()
    centers = input_helpers.sphere2center(centers,
                                    resolution=resolution,
                                    scale=scale,
                                    min_value=np.array([x[0], y[0], z[0]]))
    #pv(centers.shape)
    
    corners = coord[index[0], index[1], index[2]].reshape(-1, 8, 3)
    corners = np.array([corners[t,:,:] + centers[t,:] for t in range(corners.shape[0])])
    
    corners = bring_bbox_to_ground(corners)
    #pv(corners.shape)
    
    confidence = y_pred[index[0], index[1], index[2]]
    pv(confidence)
    
    rot = None
    if centers.shape[0] == 0:
        centers = None
        corners = None
        confidence = None
    else:
        rot = [get_orientation(corner) for corner in corners]
    
    
    results[frame_id]['pred'] = {
        'corners': corners,
        'centers': centers,
        'confidence': confidence,
        'rot': rot
    }
    #pv(rot)
    
    #label_path = input_helpers.get_label_path_for_point_path(points_path)
    #places, rots, size = input_helpers.read_labels(label_path)
    #bbox = input_helpers.get_boxcorners(places, rots, size)
    #results[frame_id]['actual'] = {
    #    'corners': bbox,
    #    'centers': places,
    #    'size': size,
    #    'rots': rots,
    #}
    
    #pc = input_helpers.load_pc_from_pcd(points_path)
    #if centers is not None:
    #    publish_pc2(pc, centers.reshape(-1, 3))#, centers[:].reshape(-1, 3))
    #else:
    #    publish_pc2(pc, np.array([[0,0,0]]))
    #break
    
pickle.dump( results, open("results_ford01.pickle", "wb") )

    

frame_id: 32
INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt
objectness.shape:  (100, 100, 10)
objectness.max && min:  -0.258145 -229.481
y_pred.shape:  (100, 100, 10)
y_pred.max && min:  0.364934 0.0
centers:  []
centers.shape:  (0, 3)
voxels_shape:  (?, 800, 800, 80, 1)
confidence: [ 0.36493438]
frame_id: 138
INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt
objectness.shape:  (100, 100, 10)
objectness.max && min:  5.80532 -305.785
y_pred.shape:  (100, 100, 10)
y_pred.max && min:  0.99999 0.0
centers:  [[65 53  4]
 [65 53  5]
 [66 53  3]
 [66 53  4]
 [66 53  5]
 [67 53  4]]
centers.shape:  (6, 3)
voxels_shape:  (?, 800, 800, 80, 1)
confidence: [ 0.99999046]
frame_id: 111
INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt
objectness.shape:  (100, 100, 10)
objectness.max &

INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt
objectness.shape:  (100, 100, 10)
objectness.max && min:  4.12826 -338.065
y_pred.shape:  (100, 100, 10)
y_pred.max && min:  0.999722 0.0
centers:  [[35 53  4]
 [36 53  4]
 [36 53  5]
 [37 53  4]]
centers.shape:  (4, 3)
voxels_shape:  (?, 800, 800, 80, 1)
confidence: [ 0.99972242]
frame_id: 246
INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt
objectness.shape:  (100, 100, 10)
objectness.max && min:  3.2232 -343.163
y_pred.shape:  (100, 100, 10)
y_pred.max && min:  0.998258 0.0
centers:  [[35 53  4]
 [35 53  5]]
centers.shape:  (2, 3)
voxels_shape:  (?, 800, 800, 80, 1)
confidence: [ 0.99825782]
frame_id: 229
INFO:tensorflow:Restoring parameters from /home/paperspace/Desktop/didi_solution/cnn_3d/3dcnn_800x800x80_res0.1_sc86.ckpt
objectness.shape:  (100, 100, 10)
objectness.max && min:  1.65574 -

In [ ]:
pv(coord.shape)
pv(centers.shape)

pv(y_pred.shape)
pv(objectness.shape)
print("-")

#label_path = input_helpers.get_label_path_for_point_path(points_path) 
pc = input_helpers.load_pc_from_pcd(points_path)
pv(pc.shape)


In [ ]:
print("cmin: ", np.min(coord) )
print("cmax: ", np.max(coord) )

print("omin: ", np.min(objectness) )
print("omax: ", np.max(objectness) )

print("ymin: ", np.min(y_pred) )
print("ymax: ", np.max(y_pred) )

In [ ]:
index = np.where(y_pred >= .95) 
index = np.array(index)
pv(index.shape)

centers = np.vstack((index[0], np.vstack((index[1], index[2])))).transpose()

centers = input_helpers.sphere2center(centers,
                                resolution=resolution,
                                scale=scale,
                                min_value=np.array([x[0], y[0], z[0]]))

pv(centers.shape)
#print(coord[0][16][85])

In [ ]:
pv(index.shape)
pv(coord.shape)
pv(index[1].shape)

In [ ]:
corners = coord[index[0], index[1], index[2]][:, :3].reshape(-1, 3) + centers[:] #[:, np.newaxis] + centers
pv(corners.shape)
pv(corners)
#pv(centers)


In [ ]:
pc = input_helpers.load_pc_from_pcd(points_path)

logic_x = np.logical_and(pc[:, 0] >= -60, pc[:, 0] < 60)
logic_y = np.logical_and(pc[:, 1] >= -60, pc[:, 1] < 60)
logic_z = np.logical_and(pc[:, 2] >= z[0], pc[:, 2] < z[1])

pc_cropped = pc[:, :3][np.logical_and(logic_x, np.logical_and(logic_y, logic_z))]

In [ ]:
pv(pc.shape)
places, rots, size = input_helpers.read_labels(label_path)
bbox = input_helpers.get_boxcorners(places, rots, size)
pv(bbox.shape)

In [ ]:
pv(corners.shape)

In [ ]:
publish_pc2(pc_cropped, corners.reshape(-1, 3), bbox.reshape(-1, 3))#, bbox.reshape(-1, 3))

In [ ]:
print(centers.shape)

In [ ]:
cn = corners.reshape(-1, 3)
pv(cn.shape)
cn_zmin = np.array([0.0, 0.0, np.max(cn[:, 2])])
pv(cn_zmin)
#cn - 
crz = cn - cn_zmin

In [ ]:
existing_pcd = [102, 142, 186, 224, 265, 308, 349, 390, 42, 70, 104, 145, 188, 227, 267, 30, 34, 392, 431, 73, 106, 147, 18, 229, 270, 311, 351, 395, 433, 75, 109, 150, 191, 22, 272, 313, 354, 397, 436, 78, 10, 152, 193, 231, 275, 315, 356, 399, 438, 80, 111, 154, 195, 234, 277, 318, 359, 39, 440, 82, 114, 157, 198, 236, 279, 320, 361, 3, 443, 85, 116, 159, 1, 239, 27, 323, 363, 402, 44, 87, 118, 15, 200, 241, 282, 325, 366, 404, 46, 8, 121, 162, 203, 243, 284, 327, 368, 407, 49, 90, 123, 164, 205, 246, 287, 32, 371, 409, 51, 92, 126, 167, 207, 248, 289, 330, 373, 412, 54, 94, 128, 169, 20, 251, 291, 332, 375, 414, 56, 97, 130, 171, 210, 253, 294, 335, 378, 416, 58, 99, 133, 174, 212, 255, 296, 337, 37, 419, 61, 135, 176, 215, 258, 299, 339, 380, 421, 63, 138, 179, 217, 25, 301, 342, 383, 424, 66, 13, 181, 219, 260, 303, 344, 385, 426, 68, 140, 183, 222, 263, 306, 347, 387, 428, 6]
labels = pt.parse_xml("/home/paperspace/Desktop/converted/car/training/nissan_driving_past_it/nissan07/tracklet_labels.xml")
print(len(existing_pcd))
print(len(results.keys()))

In [ ]:
print(labels[0])

In [ ]:
corners = objectness[0, :, :, :, 0][index].reshape(-1, 3)[:, np.newaxis] + centers
#corners2 = objectness[0, :, :, :, 1][index].reshape(-1, 3)[:, np.newaxis] + centers


In [ ]:
pv(corners.shape)

In [ ]:
publish_pc2(pc,  corners.reshape(-1, 8, 3).reshape(-1, 3))

In [11]:
import rospy
node = rospy.init_node("pc2_publisher")

In [12]:
from sensor_msgs.msg import PointCloud2
pub = rospy.Publisher("/points_raw", PointCloud2, queue_size=1000000)
pub2 = rospy.Publisher("/points_raw1", PointCloud2, queue_size=1000000)
pub3 = rospy.Publisher("/points_raw2", PointCloud2, queue_size=1000000)

In [13]:
import sensor_msgs.point_cloud2 as pc2

def create_cloud_xyzi32(header, points):
    fields = [pc2.PointField('x', 0, pc2.PointField.FLOAT32, 1),
              pc2.PointField('y', 4, pc2.PointField.FLOAT32, 1),
              pc2.PointField('z', 8, pc2.PointField.FLOAT32, 1),
              pc2.PointField('intensity', 12, pc2.PointField.FLOAT32, 1)]
    return pc2.create_cloud(header, fields, points)

def publish_pc2(pcloud, obj=None, obj2=None):
    """Publisher of PointCloud data"""
    import rospy
    from sensor_msgs.msg import PointCloud2
    import std_msgs
    from visualization_msgs.msg import Marker
    from geometry_msgs.msg import Point, TransformStamped

    r = rospy.Rate(0.1)
    
    ts = rospy.Time.now()
    
    header = std_msgs.msg.Header()
    header.stamp = ts
    header.frame_id = "velodyne"
    points = pc2.create_cloud_xyz32(header, pcloud)
    pub.publish(points)

    if obj is not None:
        header = std_msgs.msg.Header()
        header.stamp = ts
        header.frame_id = "velodyne"
        if obj.shape[-1] == 4:
            points2 = create_cloud_xyzi32(header, obj)
        else: 
            points2 = pc2.create_cloud_xyz32(header, obj)
        pub2.publish(points2)
        

    if obj2 is not None:
        header = std_msgs.msg.Header()
        header.stamp = ts
        header.frame_id = "velodyne"
        points3 = pc2.create_cloud_xyz32(header, obj2)
        pub3.publish(points3)

    #while not rospy.is_shutdown():

        
    #if obj is not None:
    #r.sleep()

In [ ]:
import sklearn
sklearn.utils.shuffle

In [ ]:
points_glob = "/home/paperspace/Desktop/converted/car/training/*/*/points/*.pcd"
labels_glob = "/home/paperspace/Desktop/converted/car/training/*/*/labels/*.pickle"

def get_label_path_for_point_path(point_path):
    ppath_parts = point_path.split('/')
    ppath_parts[-1] = ppath_parts[-1].replace('pcd', 'pickle')
    ppath_parts[-2] = 'labels'
    return os.path.join("/", *ppath_parts)
  
def visualize_points(points_path):#, label_path):
    label_path = get_label_path_for_point_path(points_path)
    pc = input_helpers.load_pc_from_pcd(points_path)
    places, rots, size = input_helpers.read_labels(label_path)

    #if(np.square(places[0]))
    place_dist = np.sqrt(np.sum(np.square(places[0])))

    if places is None or len(places.shape) == 0:
        print("places is none for: %s, %s" % (label_path, points_path))
        #continue
        return
    
    if place_dist > 50:
        print("places is too far away (%s) for: %s, %s"%(place_dist, label_path, points_path))
        #return
        #continue
    else: 
        print(points_path, place_dist)
        return True
    
    return None
        
    #corners = input_helpers.get_boxcorners(places, rots, size)
    #publish_pc2(pc, corners.reshape(-1, 3))
    #r.sleep()
    #break


points_paths = sorted( glob.glob(points_glob) )

labels_paths = sorted( glob.glob(labels_glob) )

#print(points_paths[0])
#print(labels_paths)

#visualize_points(points_paths[20])#, labels_paths[20])

#points_path = points_paths[20]
#pc = pcl.load(points_path)

#z = np.array(pc.to_list())
#z = np.array([[0, 0, 0]])
r = rospy.Rate(0.5)
for points_path in points_paths:
    if visualize_points(points_path):
        break

In [ ]:
points_paths[-1000]

In [ ]:
rospy.is_shutdown()

In [ ]:
def pc2voxel(pc, resolution=0.50, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5), car_x=(-2, 2), car_y=(-2, 2), car_z=(-3, 2)):
    """Convert PointCloud2 to Voxel"""

    # Filter out or own car :-()
    car_logic_x = np.logical_and(pc[:, 0] > car_x[0], pc[:, 0] < car_x[1])
    car_logic_y = np.logical_and(pc[:, 1] > car_y[0], pc[:, 1] < car_y[1])
    car_logic_z = np.logical_and(pc[:, 2] > car_z[0], pc[:, 2] < car_z[1])
    car_filter_xyz = np.logical_and(car_logic_x, np.logical_and(car_logic_y, car_logic_z))

    logic_x = np.logical_and(pc[:, 0] >= x[0], pc[:, 0] < x[1])
    logic_y = np.logical_and(pc[:, 1] >= y[0], pc[:, 1] < y[1])
    logic_z = np.logical_and(pc[:, 2] >= z[0], pc[:, 2] < z[1])
    logic_xyz = np.logical_and(logic_x, np.logical_and(logic_y, logic_z))

    pc = pc[:, :3][np.logical_and(np.logical_not(car_filter_xyz), logic_xyz)]
    pc = ((pc - np.array([x[0], y[0], z[0]])) / resolution).astype(np.int32)

    voxel = np.zeros((int((x[1] - x[0]) / resolution),
                      int((y[1] - y[0]) / resolution),
                      int(round((z[1] - z[0]) / resolution))
                      ))
    voxel[pc[:, 0], pc[:, 1], pc[:, 2]] = 1
    return voxel

def get_boxcorners(places, rots, size):
    """Create 8 corners of bounding box from bottom center."""
    corners = []
    for place, rot, sz in zip(places, rots, size):
        x, y, z = place
        h, w, l = sz
        if l > 10:
            continue

        corner = np.array([
            [x - l / 2.0, y - w / 2.0, z],
            [x + l / 2.0, y - w / 2.0, z],
            [x - l / 2.0, y + w / 2.0, z],
            [x - l / 2.0, y - w / 2.0, z + h],
            [x - l / 2.0, y + w / 2.0, z + h],
            [x + l / 2.0, y + w / 2.0, z],
            [x + l / 2.0, y - w / 2.0, z + h],
            [x + l / 2.0, y + w / 2.0, z + h],
        ])

        corner -= np.array([x, y, z])

        rot_matrix = np.array([
            [np.cos(rot), -np.sin(rot), 0],
            [np.sin(rot), np.cos(rot), 0],
            [0, 0, 1]
        ])

        a = np.dot(corner, rot_matrix.transpose())
        a += np.array([x, y, z])
        corners.append(a)
    return np.array(corners)


In [ ]:
def create_label(places, size, corners, resolution=0.50, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5), scale=4, min_value=None):
    """Create training Labels"""

    if min_value is None:
        #min_value = [0.0, 0.0, 0.0]
        min_value = [x[0], y[0], z[0]]
    min_value = np.array(min_value)

    places = np.array(places)
    size = np.array(size)
    corners = np.array(corners)

    x_logical = np.logical_and((places[:, 0] < x[1]), (places[:, 0] >= x[0]))
    y_logical = np.logical_and((places[:, 1] < y[1]), (places[:, 1] >= y[0]))
    z_logical = np.logical_and((places[:, 2] + size[:, 0] / 2.0 < z[1]), (places[:, 2] + size[:, 0] / 2.0 >= z[0]))
    xyz_logical = np.logical_and(x_logical, np.logical_and(y_logical, z_logical))

    center = places.copy()
    #center[:, 2] = center[:, 2] + size[:, 0] / 2.0  # Move bottom to center
    sphere_center = ((center[xyz_logical] - min_value) / (resolution * scale)).astype(np.int32)

    train_corners = corners[xyz_logical].copy()
    anchor_center = sphere2center(sphere_center,
                                     resolution=resolution,
                                     scale=scale,
                                     min_value=min_value)

    #for index, (corner, center) in enumerate(zip(corners[xyz_logical], anchor_center)):
    #    train_corners[index] = corner - center

    return sphere_center, train_corners


def sphere2center(p_sphere, resolution=0.5, scale=4, min_value=None):
    """from sphere center to label center"""

    if min_value is None:
        min_value = [0.0, 0.0, 0.0]
    min_value = np.array(min_value)

    center = p_sphere * (resolution * scale) + min_value
    return center

def corner_to_train(corners, sphere_center, resolution=0.50, x=(0, 90), y=(-50, 50), z=(-4.5, 5.5), scale=4, min_value=None):
    """Convert corner to Training label for regression loss"""

    if min_value is None:
        #min_value = [0.0, 0.0, 0.0]
        min_value = [x[0], y[0], z[0]]
    min_value = np.array(min_value)

    x_logical = np.logical_and((corners[:, :, 0] < x[1]), (corners[:, :, 0] >= x[0]))
    y_logical = np.logical_and((corners[:, :, 1] < y[1]), (corners[:, :, 1] >= y[0]))
    z_logical = np.logical_and((corners[:, :, 2] < z[1]), (corners[:, :, 2] >= z[0]))
    xyz_logical = np.logical_and(x_logical, np.logical_and(y_logical, z_logical)).all(axis=1)

    train_corners = corners[xyz_logical].copy()
    sphere_center = sphere2center(sphere_center,
                                     resolution=resolution,
                                     scale=scale,
                                     min_value=min_value)

    #for index, (corner, center) in enumerate(zip(corners[xyz_logical], sphere_center)):
        #train_corners[index] = corner - center

    return train_corners


def corner_to_voxel(voxel_shape, corners, sphere_center, scale=4):
    """Create final regression label from corner"""
    # import pdb
    # pdb.set_trace()
    corner_voxel = np.zeros((int(voxel_shape[0] / scale),
                             int(voxel_shape[1] / scale),
                             int(voxel_shape[2] / scale), 24))
    corner_voxel[sphere_center[:, 0], sphere_center[:, 1], sphere_center[:, 2]] = corners
    return corner_voxel

def create_objectness_label(sphere_center, resolution=0.5, x=90, y=100, z=10, scale=4):
    """Create Objectness label"""
    obj_maps = np.zeros((int(x / (resolution * scale)),
                         int(y / (resolution * scale)),
                         int(np.round(z / (resolution * scale)))
                         ))
    obj_maps[sphere_center[:, 0], sphere_center[:, 1], sphere_center[:, 2]] = 1
    return obj_maps



In [ ]:
points_path = "/home/paperspace/Desktop/converted/car/training/bmw_sitting_still/bmw01/points/1.pcd"
label_path = get_label_path_for_point_path(points_path) 
pc = input_helpers.load_pc_from_pcd(points_path)
places, rots, size = input_helpers.read_labels(label_path)
points = pc

In [ ]:
voxel_shape=(600, 600, 40)
resolution=0.2
scale=1
x=(-60, 60)
y=(-60, 60)
z=(-4, 4)
car_x=(-2, 2)
car_y=(-2, 2)
car_z=(-3, 2)

In [ ]:
("%sx%sx%s_"%voxel_shape)+("res%s_" % resolution)+("sc%s"%scale)

In [ ]:
corners = get_boxcorners(places, rots, size)
print("corners: ", corners.shape, corners)


In [ ]:


center_sphere, corner_label = create_label(places, size, corners,
                                           resolution=resolution,
                                           x=x,
                                           y=y,
                                           z=z,
                                           scale=scale,
                                           min_value=[x[0], y[0], z[0]])

print("center_sphere: ", center_sphere)
print("corner_label: ", corner_label.shape, corner_label)

g_map = create_objectness_label(center_sphere,
                                            resolution=resolution,
                                            x=(x[1] - x[0]),
                                            y=(y[1] - y[0]),
                                            z=(z[1] - z[0]),
                                            scale=scale)

print('g_map: ', np.argwhere(g_map > 0))

g_cord = corner_label.reshape(corner_label.shape[0], -1)
print(g_cord.shape)
g_cord = corner_to_voxel(voxel_shape, g_cord, center_sphere, scale=scale)
print(g_cord.shape)



In [ ]:
gc = np.argwhere(g_cord > 0)
gm = np.argwhere(g_map > 0)

In [ ]:
#model_01_conv.layers.cnn_model.layer2.get_shape().as_list()

print(gm.)

In [ ]:
print(gc.shape)
print(gm.shape)
print(gm)
print(center_sphere)

In [ ]:
print(gc)

In [ ]:
def nice_voxel(voxel):
    v = voxel

    v[:, 0] = v[:, 0] - np.min(v[:, 0])
    v[:, 1] = v[:, 1] - np.min(v[:, 1])
    v[:, 2] = v[:, 2] - np.min(v[:, 2])

    v[:, 0] = v[:, 0] - np.mean(v[:, 0])
    v[:, 1] = v[:, 1] - np.mean(v[:, 1])
    v[:, 2] = v[:, 2] - np.mean(v[:, 2])
    return v

In [ ]:
corner_label.shape

In [ ]:
"""Publisher of PointCloud data"""
import rospy
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2
import std_msgs
from visualization_msgs.msg import Marker
from geometry_msgs.msg import Point, TransformStamped
from sensor_msgs.msg import PointCloud2

#voxel[:, 0] = np.mean(voxel[:, 0])

#publish_pc2(v)


v = np.argwhere(pc2voxel(pc, x=x, y=y, z=z, resolution=resolution)>0)
corn = np.argwhere(pc2voxel(corners[0], x=x, y=y, z=z, resolution=resolution))

v=nice_voxel(v)
corn=nice_voxel(corn)


r = rospy.Rate(0.5)
for _ in range(2):
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = "velodyne"
    points = pc2.create_cloud_xyz32(header, v)
    pub3.publish(points)
    
    header = std_msgs.msg.Header()
    header.stamp = rospy.Time.now()
    header.frame_id = "velodyne"
    points = pc2.create_cloud_xyz32(header, corn)
    pub2.publish(points)
    
    
    #r.sleep()
